# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from tqdm.notebook import tqdm 
from sklearn.model_selection import cross_val_score
from itertools import product
from sklearn.metrics import accuracy_score

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore). Don't forget to enrich the table with the 'dayofweek' column from the previous day's .csv-file.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [4]:
df = pd.read_csv("../data/day-of-week-not-scaled.csv")
df["dayofweek"] = pd.read_csv("../data/dayofweek.csv", usecols=["dayofweek"])
df

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,dayofweek
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,9,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
1682,6,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
1683,7,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
1684,8,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3


In [5]:
x = df.drop(columns="dayofweek")
y = df["dayofweek"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [ ]:
model = SVC(random_state=21, probability=True, cache_size=10000)
param_grid = {
    "kernel": ["linear", "rbf", "sigmoid"],
    "C" : [0.01, 0.1, 1, 1.5, 5, 10],
    "gamma" : ["scale", "auto"],
    "class_weight" : ["balanced", None]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring="accuracy", n_jobs=-1)
grid_search.fit(x_train, y_train)
grid_search.best_params_

{'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}

In [5]:
grid_search.best_score_

0.8761090458488228

In [6]:
results_df = pd.DataFrame(grid_search.cv_results_)
results_df = results_df.sort_values("rank_test_score", ascending=True)
results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
70,1.698197,0.105913,0.060439,0.019256,10,None,auto,rbf,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.900000,0.848148,0.885185,0.884758,0.862454,0.876109,0.018419,1
64,2.461623,0.927569,0.060387,0.021012,10,balanced,auto,rbf,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.877778,0.851852,0.862963,0.873606,0.851301,0.863500,0.010870,2
58,2.561444,0.756749,0.110843,0.044423,5,None,auto,rbf,"{'C': 5, 'class_weight': None, 'gamma': 'auto'...",0.825926,0.811111,0.818519,0.821561,0.802974,0.816018,0.008116,3
52,2.582969,0.780783,0.066280,0.015404,5,balanced,auto,rbf,"{'C': 5, 'class_weight': 'balanced', 'gamma': ...",0.844444,0.785185,0.792593,0.817844,0.802974,0.808608,0.021007,4
63,192.491374,18.238130,0.044902,0.009481,10,balanced,auto,linear,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.729630,0.700000,0.755556,0.754647,0.665428,0.721052,0.034438,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,2.893948,1.203882,0.091643,0.040240,5,balanced,auto,sigmoid,"{'C': 5, 'class_weight': 'balanced', 'gamma': ...",0.144444,0.148148,0.137037,0.126394,0.092937,0.129792,0.019869,68
65,3.290793,1.175991,0.085700,0.041146,10,balanced,auto,sigmoid,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.122222,0.140741,0.129630,0.100372,0.085502,0.115693,0.020052,69
41,3.295684,0.816581,0.103107,0.031114,1.5,balanced,auto,sigmoid,"{'C': 1.5, 'class_weight': 'balanced', 'gamma'...",0.066667,0.085185,0.081481,0.078067,0.085502,0.079380,0.006913,70
17,5.167909,1.640234,0.077648,0.019098,0.1,balanced,auto,sigmoid,"{'C': 0.1, 'class_weight': 'balanced', 'gamma'...",0.062963,0.066667,0.062963,0.059480,0.059480,0.062310,0.002678,71


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [15]:
model = DecisionTreeClassifier()
param_grid = {
    "max_depth": range(1, 50),
    "class_weight" : ["balanced", None],
    "criterion" : ["entropy", "gini"],
    "random_state" : [21]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring="accuracy", n_jobs=-1)
grid_search.fit(x_train, y_train)
grid_search.best_params_

{'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 21,
 'random_state': 21}

In [8]:
grid_search.best_score_

0.873864794162192

In [9]:
results_df = pd.DataFrame(grid_search.cv_results_)
results_df = results_df.sort_values("rank_test_score", ascending=True)
results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
69,0.014781,0.003039,0.004113,0.000511,balanced,gini,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.859259,0.903704,0.884758,0.832714,0.873865,0.025066,1
73,0.011629,0.001085,0.004411,0.001365,balanced,gini,25,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.874074,0.903704,0.873606,0.828996,0.873854,0.025018,2
70,0.012123,0.002004,0.004949,0.001435,balanced,gini,22,"{'class_weight': 'balanced', 'criterion': 'gin...",0.885185,0.862963,0.903704,0.881041,0.828996,0.872378,0.025263,3
97,0.015450,0.002272,0.004704,0.000620,balanced,gini,49,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.828996,0.872372,0.025179,4
71,0.014080,0.002722,0.004982,0.001144,balanced,gini,23,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.828996,0.872372,0.025179,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,0.009150,0.000684,0.004772,0.000414,balanced,gini,3,"{'class_weight': 'balanced', 'criterion': 'gin...",0.388889,0.303704,0.403704,0.427509,0.345725,0.373906,0.044064,192
147,0.010557,0.002759,0.007202,0.002245,None,gini,1,"{'class_weight': None, 'criterion': 'gini', 'm...",0.370370,0.351852,0.359259,0.353160,0.342007,0.355330,0.009338,193
98,0.007594,0.000868,0.003953,0.000977,None,entropy,1,"{'class_weight': None, 'criterion': 'entropy',...",0.370370,0.351852,0.359259,0.353160,0.342007,0.355330,0.009338,193
49,0.008028,0.000377,0.006025,0.001824,balanced,gini,1,"{'class_weight': 'balanced', 'criterion': 'gin...",0.262963,0.318519,0.266667,0.323420,0.260223,0.286358,0.028376,195


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [10]:
model = RandomForestClassifier(random_state=21)
param_grid = {
    "n_estimators": [5, 10, 50, 100],
    "max_depth": range(1, 50),
    "class_weight" : ["balanced", None],
    "criterion" : ["entropy", "gini"]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring="accuracy", n_jobs=-1)
grid_search.fit(x_train, y_train)
grid_search.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 24,
 'n_estimators': 100}

In [11]:
grid_search.best_score_

0.9042929918766351

In [12]:
results_df = pd.DataFrame(grid_search.cv_results_)
results_df = results_df.sort_values("rank_test_score", ascending=True)
results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
95,0.969261,0.149674,0.079929,0.035906,balanced,entropy,24,100,"{'class_weight': 'balanced', 'criterion': 'ent...",0.922222,0.900000,0.903704,0.910781,0.884758,0.904293,0.012361,1
115,0.889944,0.147136,0.050142,0.009628,balanced,entropy,29,100,"{'class_weight': 'balanced', 'criterion': 'ent...",0.922222,0.900000,0.907407,0.907063,0.884758,0.904290,0.012156,2
698,0.446799,0.118105,0.048878,0.028310,None,gini,28,50,"{'class_weight': None, 'criterion': 'gini', 'm...",0.922222,0.900000,0.907407,0.903346,0.888476,0.904290,0.010961,2
314,0.408464,0.103415,0.026765,0.001586,balanced,gini,30,50,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.903704,0.900000,0.907063,0.884758,0.903549,0.012056,4
711,0.947566,0.269136,0.050063,0.003645,None,gini,31,100,"{'class_weight': None, 'criterion': 'gini', 'm...",0.918519,0.911111,0.900000,0.910781,0.877323,0.903547,0.014380,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,0.046749,0.019689,0.010253,0.002311,None,entropy,1,5,"{'class_weight': None, 'criterion': 'entropy',...",0.355556,0.366667,0.374074,0.345725,0.327138,0.353832,0.016467,780
4,0.033924,0.005186,0.009048,0.002172,balanced,entropy,2,5,"{'class_weight': 'balanced', 'criterion': 'ent...",0.318519,0.366667,0.381481,0.353160,0.345725,0.353110,0.021165,781
200,0.041404,0.005269,0.011105,0.005367,balanced,gini,2,5,"{'class_weight': 'balanced', 'criterion': 'gin...",0.311111,0.377778,0.377778,0.353160,0.312268,0.346419,0.029749,782
196,0.035752,0.001245,0.008997,0.000755,balanced,gini,1,5,"{'class_weight': 'balanced', 'criterion': 'gin...",0.262963,0.292593,0.285185,0.282528,0.293680,0.283390,0.011062,783


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [7]:
param_grid = {
    "n_estimators": [5, 10, 50, 100],
    "max_depth": range(1, 50),
    "class_weight" : ["balanced", None],
    "criterion" : ["entropy", "gini"]
}
param_names = list(param_grid.keys())
param_combinations = list(product(*param_grid.values()))

results = []
for params in tqdm(param_combinations):
    params_dict = dict(zip(param_names, params))
    model = RandomForestClassifier(random_state=21, n_jobs=-1, **params_dict)

    scores = cross_val_score(model, x_train, y_train, cv=5, scoring="accuracy", n_jobs=-1)
    
    results.append({
        **params_dict,
        "mean_accuracy": scores.mean(),
        "std_accuracy": scores.std()
    })

  0%|          | 0/784 [00:00<?, ?it/s]

In [11]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values("mean_accuracy", ascending=True)
results_df

,n_estimators,max_depth,class_weight,criterion,mean_accuracy,std_accuracy
0,5,1,balanced,entropy,0.270794,0.024718
1,5,1,balanced,gini,0.283390,0.011062
5,5,2,balanced,gini,0.346419,0.029749
4,5,2,balanced,entropy,0.353110,0.021165
2,5,1,None,entropy,0.353832,0.016467
...,...,...,...,...,...,...
711,100,31,None,gini,0.903547,0.014380
509,50,30,balanced,gini,0.903549,0.012056
700,100,29,balanced,entropy,0.904290,0.012156
503,50,28,None,gini,0.904290,0.010961


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [14]:
model = RandomForestClassifier(random_state=21, class_weight="balanced", criterion="entropy", max_depth=24, n_estimators=100)
model.fit(x_train, y_train)
predict = model.predict(x_test)
accuracy_score(y_test, predict)

0.9260355029585798